In [339]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline

In [40]:
plotly.tools.set_credentials_file(username='mkduchak', api_key='PQg2C65AOHoAZdL0UqLP')

In [163]:
inspections = pd.read_pickle('/home/michael/chicagohealthinspections/data/inspectdata.pkl')
locations = pd.read_pickle('/home/michael/chicagohealthinspections/data/linkloc.pkl')
businesses = pd.read_pickle('/home/michael/chicagohealthinspections/data/businesses.pkl')
category = pd.read_pickle('/home/michael/chicagohealthinspections/data/categories.pkl')
inspections['y'] = np.where(inspections['Results']=='Fail',1,0)
inspections['license'] = np.where(inspections['Inspection Type'].str.contains('License'),1,0)
inspections['prevfails'] = inspections.groupby('LocId')[['y']].cumsum()
inspections['prevfails'] = inspections['prevfails'] - inspections['y']
inspections['failpct'] = inspections['prevfails'] / inspections['inspcount']
inspections['complaint'] = np.where((inspections['Inspection Type'].str.contains('Complaint'))
                                    |(inspections['Inspection Type'].str.contains('Poisoning')),1,0)
inspections['reinspection'] = np.where(inspections['Inspection Type'].str.contains('nspection'),1,0)
businesses['price'] = [x.count('$') for x in businesses['price']]

In [122]:
neighborhoods = gpd.read_file('/home/michael/Downloads/Neighborhoods_2012/Neighborhoods_2012b.shp')
neighborhoods = neighborhoods.to_crs({'init': 'epsg:4326'})

In [388]:
data = inspections.merge(locations,
                    how='inner')
data = data.merge(businesses,
                    how='inner')

In [389]:
data.head()

,Inspection ID,License #,Facility Type,Risk,Inspection Date,Inspection Type,Results,LocId,inspcount,y,...,Address,Latitude,Longitude,id,PRI_NEIGH,name,price,rating,review_count,BusId
0,98337,60184.0,Restaurant,Risk 1 (High),2010-01-19,Suspected Food Poisoning,Pass,1,1,0,...,2829 N MILWAUKEE AVE,41.932657,-87.713056,el-ranchito-restaurant-chicago-5,Avondale,El Ranchito Restaurant,1,2.0,320.0,1
1,98414,60184.0,Restaurant,Risk 1 (High),2010-03-15,Short Form Complaint,Pass w/ Conditions,1,2,0,...,2829 N MILWAUKEE AVE,41.932657,-87.713056,el-ranchito-restaurant-chicago-5,Avondale,El Ranchito Restaurant,1,2.0,320.0,1
2,327227,60184.0,Restaurant,Risk 1 (High),2010-07-29,Complaint,Pass,1,3,0,...,2829 N MILWAUKEE AVE,41.932657,-87.713056,el-ranchito-restaurant-chicago-5,Avondale,El Ranchito Restaurant,1,2.0,320.0,1
3,365491,60184.0,Restaurant,Risk 1 (High),2011-01-11,Complaint,Fail,1,4,1,...,2829 N MILWAUKEE AVE,41.932657,-87.713056,el-ranchito-restaurant-chicago-5,Avondale,El Ranchito Restaurant,1,2.0,320.0,1
4,365507,60184.0,Restaurant,Risk 1 (High),2011-01-18,Complaint Re-Inspection,Pass,1,5,0,...,2829 N MILWAUKEE AVE,41.932657,-87.713056,el-ranchito-restaurant-chicago-5,Avondale,El Ranchito Restaurant,1,2.0,320.0,1


In [271]:
ratingcount = data[data['license']==0].groupby('rating',as_index=False)[['y']].count()
ratingcount.columns = ['rating', 'count']
ratingsum = data[data['license']==0].groupby('rating',as_index=False)[['y']].sum()
ratingsum.columns = ['rating', 'fail']
ratingdf = ratingcount.merge(ratingsum,how='left')
ratingdf['pass'] = ratingdf['count'] - ratingdf['fail']
ratingdf['failpct'] = (round(ratingdf['fail']/ratingdf['count'],3)*100).astype(str)+'%'
ratingdf

,rating,count,fail,pass,failpct
0,1.0,787,177,610,22.5%
1,1.5,1632,299,1333,18.3%
2,2.0,3389,673,2716,19.9%
3,2.5,7189,1363,5826,19.0%
4,3.0,12321,2580,9741,20.9%
5,3.5,20616,4180,16436,20.3%
6,4.0,18882,3449,15433,18.3%
7,4.5,6317,1101,5216,17.4%
8,5.0,1151,194,957,16.9%


In [277]:
bar = [
    go.Bar(
        name = 'Failed',
        x=ratingdf['rating'],
        y=ratingdf['fail'],
        marker=dict(
            color='rgb(230, 158, 12)')
            ),
    go.Bar(
        name = 'Passed',
        x=ratingdf['rating'],
        y=ratingdf['pass'],
        marker=dict(
            color='rgb(80,138,23)')
            ),
    go.Scatter(
        name = 'Failure Pct',
        x=ratingdf['rating'],
        y=ratingdf['failpct'],
        showlegend=False,
        yaxis='y2',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 4),
        mode='lines+text',
        text=ratingdf['failpct'],
        textposition='top right',
        textfont=dict(
                size=16
                )
            )
        ]


layout = go.Layout(
    barmode='stack',
    title='Fail Rate by Rating',
    titlefont=dict(size=20),
    legend=dict(
        font=dict(
            size=20)
        ),
    yaxis=dict(
        title='Inspections',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
        ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        showgrid=False,
        showticklabels=False,
        range=[0, 25]
        ),
    xaxis=dict(tickfont=dict(size=18),title='Rating',titlefont=dict(size=20))
)



fig1 = go.Figure(data=bar, layout=layout)


py.iplot(fig1)

In [390]:
worstbusinesses = data[data['license']==0].sort_values('prevfails',ascending=False).head(50)
worstbusinesses = worstbusinesses.drop_duplicates(subset='name')
worstbusinesses = worstbusinesses[['name','PRI_NEIGH','prevfails','failpct','rating']].head(10)
worstbusinesses.columns = ['Name','Neighborhood','Failures','% of Failed Inspections','Rating']
worstbusinesses

,Name,Neighborhood,Failures,% of Failed Inspections,Rating
27299,Soho House,West Loop,16,0.421053,3.5
49029,Jimmy G's Restaurant,Garfield Park,15,0.416667,4.5
38139,China Cafe,Chinatown,14,0.560000,3.0
28520,McDonald's,Washington Park,13,0.333333,1.0
34113,Congress Pizzeria,Logan Square,13,0.433333,2.5
23799,Food 4 Less,Englewood,13,0.295455,2.5
8203,MingHin Cuisine,Chinatown,13,0.382353,4.0
902,Pizza Capri,Hyde Park,12,0.400000,3.5
24366,Latin American Restaurant & Lounge,Humboldt Park,12,0.413793,2.5
7632,Mariscos Las Islitas,Lower West Side,12,0.545455,3.0


In [152]:
totalpct = data[data['license']==0].groupby(['name','PRI_NEIGH'],as_index=False)[['prevfails','inspcount']].max()
totalpct['failpct'] = round(totalpct['prevfails']/totalpct['inspcount'],3)
totalpct.sort_values('failpct',ascending=False).head(10)

,name,PRI_NEIGH,prevfails,inspcount,failpct
3251,Green Olive,Hegewisch,3,4,0.750
5301,Mr Philly & More,Austin,3,4,0.750
8529,West Town Tavern,West Town,3,4,0.750
2048,Dee's Place,Wicker Park,5,7,0.714
3206,Grand Katachi,Lincoln Square,9,13,0.692
8066,Today's Temptation,Galewood,4,6,0.667
2936,Fresco Sweets & Snacks,West Ridge,4,6,0.667
6605,Royal Chicken & Seafood,Austin,4,6,0.667
1139,CVS Pharmacy,Logan Square,2,3,0.667
7770,The Cupcake Gallery,Uptown,8,12,0.667


In [155]:
totalpct.sort_values(['prevfails','inspcount'],ascending=[True,False]).head(10)

,name,PRI_NEIGH,prevfails,inspcount,failpct
5744,Paleteria Bambi,New City,0,27,0.0
1327,Carbon,Armour Square,0,20,0.0
3898,Jewel-Osco,West Town,0,18,0.0
4874,Manny's,Garfield Ridge,0,18,0.0
3412,Harry Caray's Restaurant,Garfield Ridge,0,17,0.0
7326,Susie's,Irving Park,0,17,0.0
930,Bread Basket,Belmont Cragin,0,16,0.0
6531,Robinson's No. 1 Ribs,West Loop,0,16,0.0
7908,The Original Pancake House,Beverly,0,16,0.0
191,Alaska Paleteria Y Neveria,Irving Park,0,15,0.0


In [251]:
totalpct['failind'] = np.where(totalpct['prevfails']==0,0,1)
restfails = totalpct.groupby('failind')[['name']].count()
restfails.iloc[1]/(restfails.iloc[0]+restfails.iloc[1])

name    0.71815
dtype: float64

In [252]:
restfails

,name
failind,
0,2480
1,6319


In [307]:
hist_data = [totalpct['failpct']]
group_labels = ['Restaurants']

fig = ff.create_distplot(hist_data, group_labels,bin_size=.05,colors=['#E89E0C'])
fig['layout'].update(title='Restaurant Inspection Failure Percentage')
iplot(fig)

In [229]:
pricecount = data[data['license']==0].groupby('price',as_index=False)[['y']].count()
pricecount.columns = ['price', 'count']
pricesum = data[data['license']==0].groupby('price',as_index=False)[['y']].sum()
pricesum.columns = ['price', 'fail']
pricedf = pricecount.merge(pricesum,how='left')
pricedf['pass'] = pricedf['count'] - pricedf['fail']
pricedf['failpct'] = (round(pricedf['fail']/pricedf['count'],3)*100).astype(str)+'%'
pricedf

,price,count,fail,pass,failpct
0,1,37583,7062,30521,18.8%
1,2,30531,6191,24340,20.3%
2,3,3566,649,2917,18.2%
3,4,604,114,490,18.9%


In [230]:
bar = [
    go.Bar(
        name = 'Failed',
        x=pricedf['price'],
        y=pricedf['fail'],
        marker=dict(
            color='rgb(230, 158, 12)')
            ),
    go.Bar(
        name = 'Passed',
        x=pricedf['price'],
        y=pricedf['pass'],
        marker=dict(
            color='rgb(80,138,23)')
            ),
    go.Scatter(
        name = 'Failure Pct',
        x=pricedf['price'],
        y=pricedf['failpct'],
        showlegend=False,
        yaxis='y2',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 4),
        mode='lines+text',
        text=pricedf['failpct'],
        textposition='top',
        textfont=dict(
                size=18
                )
            )
        ]


layout = go.Layout(
    barmode='stack',
    title='Fail Rate by Price Level',
    titlefont=dict(size=20),
    legend=dict(
        font=dict(
            size=20)
        ),
    yaxis=dict(
        title='Inspections',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
        ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        showgrid=False,
        showticklabels=False,
        range=[0, 25]
        ),
    xaxis=dict(
        range=[0.5, 4.5],
        tick0=0,
        dtick=1,
        ticklen=4,
        tickfont=dict(size=18)
    )
)



fig1 = go.Figure(data=bar, layout=layout)


py.iplot(fig1)

In [205]:
resultdf = data.groupby('Results',as_index=False)[['y']].count()
resultdf.columns = ['Results', 'Count']
resultdf['Percent'] = round(resultdf['Count']/resultdf['Count'].sum(),3)*100
resultdf['Percent'] = resultdf['Percent'].astype(str) + '%'

resultdf

,Results,Count,Percent
0,Fail,17752,21.1%
1,Pass,55983,66.5%
2,Pass w/ Conditions,10483,12.4%


In [375]:
bar = [
    go.Bar(
        name = 'Inspection Count',
        x=resultdf['Results'],
        y=resultdf['Count'],
        text=resultdf['Percent'],
        textposition = 'auto',
        textfont=dict(
                size=20,
                color='rgb(255, 255, 255)'
                ),
        marker=dict(
            color='rgb(80,138,23)')
            )
  
        ]


layout = go.Layout(
    barmode='bar',
    title='Inspection Results Summary',
    titlefont=dict(
        size = 20),
    xaxis=dict(
        tickfont=dict(
            size=18)
        ),
    yaxis=dict(
        title='Inspections',
        titlefont=dict(
            size = 18),
        tickfont=dict(
            size=18)
        )
    )
    




fig1 = go.Figure(data=bar, layout=layout)


py.iplot(fig1)

In [383]:
insptype = data[['Inspection Type','y','license','complaint','reinspection']].copy(deep=True)
insptype['canvass'] = np.where(insptype['Inspection Type'].str.contains('Canvass'),1,0)
insptype['other'] = np.where(((insptype['canvass']==0)&(insptype['license']==0)&(insptype['complaint']==0)),1,0)
insptype['insptype'] = insptype[['canvass','license','complaint','other']].idxmax(axis=1)
insptypecount = insptype.groupby('insptype',as_index=False)[['y']].count()
insptypecount.columns = ['insptype', 'count']
insptypesum = insptype.groupby('insptype',as_index=False)[['y']].sum()
insptypesum.columns = ['insptype', 'fail']
insptypedf = insptypecount.merge(insptypesum, how = 'inner')
insptypedf['pass'] = insptypedf['count'] - insptypedf['fail']
insptypedf['failpct'] = (round(insptypedf['fail']/insptypedf['count'],3)*100).astype(str)+'%'
insptypedf['pctoffail'] = (round(insptypedf['fail']/sum(insptypedf['fail']),3)*100).astype(str)+'%'
insptypedf

,insptype,count,fail,pass,failpct,pctoffail
0,canvass,51402,9341,42061,18.2%,52.6%
1,complaint,19532,4382,15150,22.4%,24.7%
2,license,11933,3736,8197,31.3%,21.0%
3,other,1351,293,1058,21.7%,1.7%


In [384]:
bar = [
    go.Bar(
        name = 'Failed',
        x=insptypedf['insptype'],
        y=insptypedf['fail'],
        marker=dict(
            color='rgb(230, 158, 12)')
            ),
    go.Bar(
        name = 'Passed',
        x=insptypedf['insptype'],
        y=insptypedf['pass'],
        marker=dict(
            color='rgb(80,138,23)')
            ),
    go.Scatter(
        name = 'Failure Pct',
        x=insptypedf['insptype'],
        y=insptypedf['failpct'],
        showlegend=False,
        yaxis='y2',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 4),
        mode='lines+text',
        text=insptypedf['failpct'],
        textposition='top',
        textfont=dict(
                size=18
                )
            )
        ]


layout = go.Layout(
    barmode='stack',
    title='Inspection Type Summary',
    titlefont=dict(size=20),
    legend=dict(
        font=dict(
            size=20)
        ),
    yaxis=dict(
        title='Total Inspections',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
        ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        showgrid=False,
        showticklabels=False,
        range=[0, 40]
        ),
    xaxis=dict(
        title='Inspection Type',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
    )
)



fig1 = go.Figure(data=bar, layout=layout)


py.iplot(fig1)

In [395]:

reinspcount = insptype.groupby('reinspection',as_index=False)[['y']].count()
reinspcount.columns = ['reinspection', 'count']
reinspsum = insptype.groupby('reinspection',as_index=False)[['y']].sum()
reinspsum.columns = ['reinspection', 'fail']
reinsp = reinspcount.merge(reinspsum, how = 'inner')
reinsp['pass'] = reinsp['count'] - reinsp['fail']
reinsp['failpct'] = (round(reinsp['fail']/reinsp['count'],3)*100).astype(str)+'%'
reinsp['insp'] = np.where(reinsp['reinspection']==0,'Normal Inspection','ReInspection')
reinsp

,reinspection,count,fail,pass,failpct,insp
0,0,66466,16611,49855,25.0%,Normal Inspection
1,1,17752,1141,16611,6.4%,ReInspection


In [397]:
bar = [
    go.Bar(
        name = 'Failed',
        x=reinsp['insp'],
        y=reinsp['fail'],
        marker=dict(
            color='rgb(230, 158, 12)')
            ),
    go.Bar(
        name = 'Passed',
        x=reinsp['insp'],
        y=reinsp['pass'],
        marker=dict(
            color='rgb(80,138,23)')
            ),
    go.Scatter(
        name = 'Failure Pct',
        x=reinsp['insp'],
        y=reinsp['failpct'],
        showlegend=False,
        yaxis='y2',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 4),
        mode='lines+text',
        text=reinsp['failpct'],
        textposition='top',
        textfont=dict(
                size=18
                )
            )
        ]


layout = go.Layout(
    barmode='stack',
    title='Inspection Type Summary',
    titlefont=dict(size=20),
    legend=dict(
        font=dict(
            size=20)
        ),
    yaxis=dict(
        title='Total Inspections',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
        ),
    yaxis2=dict(
        overlaying='y',
        side='right',
        showgrid=False,
        showticklabels=False,
        range=[0, 40]
        ),
    xaxis=dict(
        title='Inspection Type',
        titlefont=dict(size=20),
        tickfont=dict(size=18)
    )
)



fig1 = go.Figure(data=bar, layout=layout)


py.iplot(fig1)

In [349]:
catdf = category[['id','title']].merge(locations[['id','LocId']],how='inner')
catdf = catdf[['title','LocId']].merge(inspections[['LocId','Inspection ID','y']])
catcount = catdf.groupby('title',as_index=False)[['y']].count()
catcount.columns = ['title', 'count']
catsum = catdf.groupby('title',as_index=False)[['y']].sum()
catsum.columns = ['title', 'fail']
cattotal = catcount.merge(catsum, how = 'inner')
cattotal['pass'] = cattotal['count'] - cattotal['fail']
cattotal['failpct'] = (round(cattotal['fail']/cattotal['count'],3)*100).astype(str)+'%'
cattotal['pctoffail'] = (round(cattotal['fail']/sum(cattotal['fail']),3)*100).astype(str)+'%'
cat10 = cattotal.sort_values('count',ascending=False).head(15)
cat10

,title,count,fail,pass,failpct,pctoffail
277,Mexican,9817,1984,7833,20.2%,5.5%
370,Sandwiches,9429,1631,7798,17.3%,4.5%
171,Fast Food,9324,1626,7698,17.4%,4.5%
333,Pizza,6784,1417,5367,20.9%,3.9%
107,Coffee & Tea,6524,1233,5291,18.9%,3.4%
14,American (Traditional),6390,1464,4926,22.9%,4.0%
75,Burgers,5649,1119,4530,19.8%,3.1%
67,Breakfast & Brunch,5532,1175,4357,21.2%,3.2%
201,Grocery,5222,1091,4131,20.9%,3.0%
100,Chinese,4690,1077,3613,23.0%,3.0%


In [386]:
trace1 = {"x": cat10['title'], 
          "y": cat10['count'], 
          "marker": {"color": 'rgb(80,138,23)', "size": 16}, 
          "mode": "markers", 
          "name": "Total Inspections", 
          "type": "scatter"
}

trace2 = {"x": cat10['title'], 
          "y": cat10['fail'], 
          "marker": {"color":'rgb(230, 158, 12)', "size": 16}, 
          "mode": "markers", 
          "name": "Failed Inspections", 
          "type": "scatter", 
}
trace3 = go.Scatter(
                name = 'Failure Pct',
                x=cat10['title'],
                y=cat10['failpct'],
                showlegend=True,
                yaxis='y2',
                line = dict(
                        color = ('rgb(255, 0, 0)'),
                        width = 6),
                mode='markers+text',
                text=cat10['failpct'],
                textposition='top',
                textfont=dict(size=16)
)

data = go.Data([trace1, trace2, trace3])
layout = {"title": "Inspections by Category", 
          "yaxis": {"title": "Inspections"},
          "yaxis2": {'overlaying':'y',
                        'side':'right',
                        'showgrid':False,
                        'showticklabels':False,
                        'range':[0, 30]},
          "xaxis": {'tickfont':{'size':14}}
          
         }

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)